In [1]:
import numpy as np
import pandas as pd

import sys
sys.path.append('../')

from Modules.Utils.Imports import *
from Modules.Models.BuildBINNs import BINNCovasim
from Modules.Utils.ModelWrapper import ModelWrapper

import Modules.Utils.PDESolver as PDESolver
import Modules.Loaders.DataFormatter as DF
from utils import get_case_name_drums, get_case_name, AIC_OLS, RSS

c:\Users\Teddy\anaconda3\envs\reu_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# helper functions
def to_torch(x):
    return torch.from_numpy(x).float().to(device)
def to_numpy(x):
    return x.detach().cpu().numpy()

In [3]:
'''
device = torch.device(GetLowestGPU(pick_from=[0,1,2,3]))
# choose and get data
path = '../Data/covasim_data/drums_data/'
population = 200000
test_prob = 0.1
trace_prob = 0.3
keep_d = True
retrain = False
dynamic = True
chi_type = 'piecewise'
date = '0623_1017'
case_name = get_case_name_drums(date)
params = DF.load_covasim_data(path, population, test_prob, trace_prob, keep_d, case_name, plot=False)
'''

"\ndevice = torch.device(GetLowestGPU(pick_from=[0,1,2,3]))\n# choose and get data\npath = '../Data/covasim_data/drums_data/'\npopulation = 200000\ntest_prob = 0.1\ntrace_prob = 0.3\nkeep_d = True\nretrain = False\ndynamic = True\nchi_type = 'piecewise'\ndate = '0623_1017'\ncase_name = get_case_name_drums(date)\nparams = DF.load_covasim_data(path, population, test_prob, trace_prob, keep_d, case_name, plot=False)\n"

In [4]:
'''
comps = list(params.keys())[1:]
for comp in comps:
    if comp=='S':
        data = np.array(params[comp])[:,None]
    else:
        new_col = np.array(params[comp])[:,None]
        data = np.append(data, new_col, 1)
'''

"\ncomps = list(params.keys())[1:]\nfor comp in comps:\n    if comp=='S':\n        data = np.array(params[comp])[:,None]\n    else:\n        new_col = np.array(params[comp])[:,None]\n        data = np.append(data, new_col, 1)\n"

In [5]:
'''
data = (data / 50000.)
N = len(data)
t_max = N - 1
t = np.arange(N)
'''

'\ndata = (data / 50000.)\nN = len(data)\nt_max = N - 1\nt = np.arange(N)\n'

In [6]:
# instantiate BINN model
# # binn = BINNCovasim(params, t_max, tracing_array=None, keep_d=keep_d).to(device)

In [7]:
device = torch.device(GetLowestGPU(pick_from=[0,1,2,3]))

# choose and get data
path = '../Data/covasim_data/drums_data/'
population = 200000
test_prob = 0.1
trace_prob = 0.3
keep_d = True
retrain = False
dynamic = True
chi_type = 'linear'

case_name = get_case_name(population, test_prob, trace_prob, keep_d, dynamic=dynamic, chi_type=chi_type)
params = DF.load_covasim_data(path, population, test_prob, trace_prob, keep_d, case_name, plot=False)

Device set to cpu


In [8]:
data = params['data']
data = (data / params['population']).to_numpy()
N = len(data)
t_max = N - 1
t = np.arange(N)
params.pop('data')

,S,T,E,A,Y,D,Q,R,F
0,199600,0,400,0,0,0,0,0,0
1,199599,0,400,0,1,0,0,0,0
2,199593,0,389,5,13,0,0,0,0
3,199559,0,313,38,88,2,0,0,0
4,199480,0,290,69,150,11,0,0,0
...,...,...,...,...,...,...,...,...,...
178,104807,0,1137,808,1070,736,0,90955,487
179,104628,0,1130,791,1058,713,0,91192,488
180,104481,0,1122,783,1023,689,0,91410,492
181,104309,0,1108,778,1038,672,0,91601,494


In [9]:
tracing_array = params['tracing_array'] # tracing array - h(t) for each t from 1 to 200

mydir = '../models/covasim/2023-06-21_00-12-29' # BINN trained on data with piecewise h(t) function

In [15]:
# instantiate BINN model
binn = BINNCovasim(params, t_max, tracing_array, keep_d=keep_d).to(device)
print(case_name)

200000_0.1_0.3_dynamic_linear


In [13]:
parameters = binn.parameters()
model = ModelWrapper(binn, None, None, save_name=os.path.join(mydir, '200000_0.1_0.3_dynamic_piecewise'))

In [14]:
# load model weights. if retrain==True then load the retrained model
if retrain:
    model.save_name += '_retrain'
model.save_name += '_best_val'
model.load(model.save_name + '_model', device=device)

# grab initial condition
u0 = data[0, :].copy()

FileNotFoundError: [Errno 2] No such file or directory: '../models/covasim/2023-06-21_00-12-29\\200000_0.1_0.3_dynamic_linear_best_val_model'